In [21]:
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
bucket = 'sxm-ecomm-sagemaker-dev'
container = {'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:latest',}
training_image = container[boto3.Session().region_name]
print(training_image)

arn:aws:iam::969580840594:role/sxm-ecomm-sagemaker-role-dev
811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:latest


In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [23]:
data = pd.read_csv('data/road_or_otherNEW.csv')
#data = data.drop(['weird_imgs'], axis =1)
data = data.drop(data.columns[0], axis=1)
data.head()

,img,type,path
0,img_21.331533_-157.865814_freq_1,0,sxm-ecomm-sagemaker-dev/freqs/img_21.331533_-1...
1,img_21.331533_-157.865814_freq_2,0,sxm-ecomm-sagemaker-dev/freqs/img_21.331533_-1...
2,img_21.331533_-157.865814_freq_3,1,sxm-ecomm-sagemaker-dev/freqs/img_21.331533_-1...
3,img_21.331533_-157.865814_freq_4,0,sxm-ecomm-sagemaker-dev/freqs/img_21.331533_-1...
4,img_28.782888_-97.045563_freq_1,0,sxm-ecomm-sagemaker-dev/freqs/img_28.782888_-9...


In [24]:
data['path2'] = [str(path).replace('sxm-ecomm-sagemaker-dev/freqs/', '') for path in data['path']]

# data['path'] = 'road' if data['type'] == 1 else 'other'
# str = "%s/%s" % ('road' if type == 1 else 'other', path)
# X.head(1)[0]

data.loc[data['type'] == 0, 'path'] = 'other/' + data['path2']
data.loc[data['type'] == 1, 'path'] = 'road/' + data['path2']

X = data.path
y = data.type

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True)

In [26]:
X_train.head()

91      other/img_34.311032_-92.413717_freq_4.png
0      other/img_21.331533_-157.865814_freq_1.png
330      road/img_40.826027_-91.208710_freq_3.png
124     other/img_35.463771_-97.417655_freq_1.png
187    other/img_37.568253_-122.026146_freq_4.png
Name: path, dtype: object

In [27]:
y_train.head()

91     0
0      0
330    1
124    0
187    0
Name: type, dtype: int64

In [28]:
result_train = pd.concat([y_train, X_train], axis=1, join="inner")


In [29]:
result_train.reset_index(inplace=True)

In [30]:
result_train

,index,type,path
0,91,0,other/img_34.311032_-92.413717_freq_4.png
1,0,0,other/img_21.331533_-157.865814_freq_1.png
2,330,1,road/img_40.826027_-91.208710_freq_3.png
3,124,0,other/img_35.463771_-97.417655_freq_1.png
4,187,0,other/img_37.568253_-122.026146_freq_4.png
...,...,...,...
315,39,0,other/img_32.196769_-95.853902_freq_4.png
316,237,1,road/img_38.685054_-90.339324_freq_2.png
317,154,0,other/img_36.168284_-95.922185_freq_3.png
318,340,0,other/img_41.651033_-86.216179_freq_1.png


In [31]:
result_train.to_csv('trainingNEW.lst', header=None, index=None, sep='\t')

In [32]:
result_train

,index,type,path
0,91,0,other/img_34.311032_-92.413717_freq_4.png
1,0,0,other/img_21.331533_-157.865814_freq_1.png
2,330,1,road/img_40.826027_-91.208710_freq_3.png
3,124,0,other/img_35.463771_-97.417655_freq_1.png
4,187,0,other/img_37.568253_-122.026146_freq_4.png
...,...,...,...
315,39,0,other/img_32.196769_-95.853902_freq_4.png
316,237,1,road/img_38.685054_-90.339324_freq_2.png
317,154,0,other/img_36.168284_-95.922185_freq_3.png
318,340,0,other/img_41.651033_-86.216179_freq_1.png


In [33]:
result_test = pd.concat([y_test, X_test], axis=1, join="inner")
result_test.reset_index(inplace=True)
result_test.to_csv('validationNEW.lst', header=None, index=None, sep='\t')

In [34]:
import pandas as pd
import numpy as np

testing = pd.read_csv('validationNEW.lst', sep='\t', header=None, names = ['index', 'label', 'path'])
training = pd.read_csv('trainingNEW.lst', sep='\t', header=None, names = ['index', 'label', 'path'])


In [35]:
testing

,index,label,path
0,393,0,other/img_47.138551_-122.525624_freq_2.png
1,200,1,road/img_37.761433_-122.477325_freq_1.png
2,312,0,other/img_40.364099_-111.742177_freq_1.png
3,198,1,road/img_37.754088_-100.044436_freq_3.png
4,314,1,road/img_40.364099_-111.742177_freq_3.png
...,...,...,...
75,239,0,other/img_38.685054_-90.339324_freq_4.png
76,247,0,other/img_38.709484_-90.880325_freq_4.png
77,229,0,other/img_38.567398_-90.495452_freq_2.png
78,272,0,other/img_39.173099_-91.886520_freq_1.png


In [36]:
s3_resource = boto3.resource('s3')
# Move files into correct folders


# Copy object A as object B
bucket = 'sxm-ecomm-sagemaker-dev'

for ind in testing.index:
    path = testing['path'][ind]
    label = testing['label'][ind]
    index = testing['index'][ind]
    oldPath = path.replace('other', 'freqs')
    oldPath = oldPath.replace('road', 'freqs')
    print(path)
    print(oldPath)
    print(index)
    
    copy_source = {
        'Bucket': bucket,
        'Key': oldPath
    }
    print("image-classification/validation/%s" % (path))
    response = s3_resource.meta.client.copy(copy_source, bucket, "image-classification/validation/%s" % (path))
    print(response)



other/img_47.138551_-122.525624_freq_2.png
freqs/img_47.138551_-122.525624_freq_2.png
393
image-classification/validation/other/img_47.138551_-122.525624_freq_2.png
None
road/img_37.761433_-122.477325_freq_1.png
freqs/img_37.761433_-122.477325_freq_1.png
200
image-classification/validation/road/img_37.761433_-122.477325_freq_1.png
None
other/img_40.364099_-111.742177_freq_1.png
freqs/img_40.364099_-111.742177_freq_1.png
312
image-classification/validation/other/img_40.364099_-111.742177_freq_1.png
None
road/img_37.754088_-100.044436_freq_3.png
freqs/img_37.754088_-100.044436_freq_3.png
198
image-classification/validation/road/img_37.754088_-100.044436_freq_3.png
None
road/img_40.364099_-111.742177_freq_3.png
freqs/img_40.364099_-111.742177_freq_3.png
314
image-classification/validation/road/img_40.364099_-111.742177_freq_3.png
None
other/img_35.463771_-97.417655_freq_4.png
freqs/img_35.463771_-97.417655_freq_4.png
127
image-classification/validation/other/img_35.463771_-97.417655_freq_

In [38]:
s3_resource = boto3.resource('s3')
# Move files into correct folders


# Copy object A as object B
bucket = 'sxm-ecomm-sagemaker-dev'

for ind in training.index:
    path = training['path'][ind]
    label = training['label'][ind]
    index = training['index'][ind]
    oldPath = path.replace('other', 'freqs')
    oldPath = oldPath.replace('road', 'freqs')
    print(path)
    print(oldPath)
    print(index)
    
    copy_source = {
        'Bucket': bucket,
        'Key': oldPath
    }
    response = s3_resource.meta.client.copy(copy_source, bucket, "image-classification/training/%s" % (path))
    print(response)


other/img_34.311032_-92.413717_freq_4.png
freqs/img_34.311032_-92.413717_freq_4.png
91
None
other/img_21.331533_-157.865814_freq_1.png
freqs/img_21.331533_-157.865814_freq_1.png
0
None
road/img_40.826027_-91.208710_freq_3.png
freqs/img_40.826027_-91.208710_freq_3.png
330
None
other/img_35.463771_-97.417655_freq_1.png
freqs/img_35.463771_-97.417655_freq_1.png
124
None
other/img_37.568253_-122.026146_freq_4.png
freqs/img_37.568253_-122.026146_freq_4.png
187
None
other/img_35.484282_-120.665052_freq_4.png
freqs/img_35.484282_-120.665052_freq_4.png
139
None
road/img_30.742342_-99.225417_freq_2.png
freqs/img_30.742342_-99.225417_freq_2.png
21
None
other/img_45.504331_-122.475114_freq_3.png
freqs/img_45.504331_-122.475114_freq_3.png
378
None
other/img_32.781604_-97.388521_freq_4.png
freqs/img_32.781604_-97.388521_freq_4.png
51
None
road/img_37.315876_-122.031860_freq_3.png
freqs/img_37.315876_-122.031860_freq_3.png
182
None
other/img_35.484282_-120.665052_freq_2.png
freqs/img_35.484282_-120.